In [1]:
# import modules 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import csv
import filters as f

Linking to databases

In [2]:
# Path to data file in csv format
data_file = "/home/gfemer/projects/ctb-sgravel/cartagene/research/quebec_structure_936028/data/old_metadata/data_Gravel936028_2.csv"
# Path to metadata info file in xlsx format
catalog_file = '/home/gfemer/projects/ctb-sgravel/cartagene/research/quebec_structure_936028/data/old_metadata/COMBINED_CATALOG_v2_9_6 - JUIL2020_0.xlsx'
# Path to phase2 country other codes
phase2_file = "/home/gfemer/storage/Cartagene/databases/phase2_country.txt"
# Path to ISO county code file
iso_file = "/home/gfemer/storage/Cartagene/databases/ISO_country_codes_edited_cartagene.txt"

In [3]:
# Relevant fields
fields = "ethnic,country"
fields = "file111,PHASE,CREGION,"+fields

In [4]:
data = f.parse_metadata(data_file,catalog_file,fields,drop_other=False)
data = f.correct_iso(data,phase2_file,iso_file,drop_other=True)

Reading metadata
Finished reading data


In [42]:
data.keys()

Index(['file111', 'phase', 'COUNTRY_BIRTH', 'MOTHER_COUNTRY_BIRTH',
       'FATHER_COUNTRY_BIRTH', 'MOTHERS_MOTHER_COUNTRY_BIRTH',
       'MOTHERS_FATHER_COUNTRY_BIRTH', 'FATHERS_MOTHER_COUNTRY_BIRTH',
       'FATHERS_FATHER_COUNTRY_BIRTH'],
      dtype='object')

In [14]:
data[data.ETHNICITY_ME == "ABORIGINAL"].groupby("COUNTRY_BIRTH").ETHNICITY_ME.value_counts()

COUNTRY_BIRTH  ETHNICITY_ME
CANADA         ABORIGINAL      63
PERU           ABORIGINAL       2
Name: ETHNICITY_ME, dtype: int64

In [15]:
data[data.ETHNICITY_ME == "WHITEABORIGINAL"].groupby("COUNTRY_BIRTH").ETHNICITY_ME.value_counts()

COUNTRY_BIRTH  ETHNICITY_ME   
CANADA         WHITEABORIGINAL    182
UNITED_STATES  WHITEABORIGINAL      1
Name: ETHNICITY_ME, dtype: int64

In [5]:
# Relevant filter and sample
filters = "CANADA"
filters = filters.split(",")

In [6]:
# Check for Mixed Ancestry in the grandparents
NA_data= pd.DataFrame()
expression='MOTHERS|FATHERS'

data_all = f.filter_by_value(data,filters)

Count number of grandparents in country = Canada

In [7]:
# Find columns with 'MOTHERS' or 'FATHERS' in their names
filtered_columns = [col for col in data_all.columns if 'MOTHERS' in col or 'FATHERS' in col]

data_all['NGP_COUNTRY_INTEREST'] = data_all[filtered_columns].isin(filters).sum(axis=1)

print("Total individuals extracted ", len(data_all))

# Extract individuals with total ancestry
#data_med_totalancs = data_all[data_all.NGP_COUNTRY_INTEREST == 4].copy()

# Removing individuals with none to only one grandparent in the countries of interest
data_med = data_all[data_all.NGP_COUNTRY_INTEREST > 1].copy()

data_med_totalancs = data_med[data_med[filtered_columns].nunique(axis=1) == 1]

#data_all[data_all.NGP_COUNTRY_INTEREST == 4].copy()

data_med_totalancs['Population'] = data_med_totalancs.MOTHERS_MOTHER_COUNTRY_BIRTH
print("Total inds. with full ancestry: ",len(data_med_totalancs))

print("Total indvs. with at least two grandparents born in the countries of interest ", len(data_med))
print("0 Granparents: ", len(data_all[data_all.NGP_COUNTRY_INTEREST == 0]))
print("1 Granparent: ", len(data_all[data_all.NGP_COUNTRY_INTEREST == 1]))
print("2 Granparents: ", len(data_all[data_all.NGP_COUNTRY_INTEREST == 2]))
print("3 Granparents: ", len(data_all[data_all.NGP_COUNTRY_INTEREST == 3]))
print("4 Granparents: ", len(data_all[data_all.NGP_COUNTRY_INTEREST == 4]))
print("4 Grandparents born in the same country",(data_med[filtered_columns].nunique(axis=1) == 1).sum())

print("Total retained after removing 0-1 gps: ", len(data_med))

Total individuals extracted  25291


/tmp/ipykernel_2547061/2171568480.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_med_totalancs['Population'] = data_med_totalancs.MOTHERS_MOTHER_COUNTRY_BIRTH


Total inds. with full ancestry:  20628
Total indvs. with at least two grandparents born in the countries of interest  23719
0 Granparents:  1318
1 Granparent:  254
2 Granparents:  1361
3 Granparents:  1730
4 Granparents:  20628
4 Grandparents born in the same country 20628
Total retained after removing 0-1 gps:  23719


In [8]:
data_med = data_med[data_med[filtered_columns].nunique(axis=1) == 1].copy()

In [9]:
# Extract individuals with total ancestry
data_med['Population'] = data_med.MOTHERS_MOTHER_COUNTRY_BIRTH

In [10]:
data_med

,file111,phase,COUNTRY_BIRTH,MOTHER_COUNTRY_BIRTH,FATHER_COUNTRY_BIRTH,MOTHERS_MOTHER_COUNTRY_BIRTH,MOTHERS_FATHER_COUNTRY_BIRTH,FATHERS_MOTHER_COUNTRY_BIRTH,FATHERS_FATHER_COUNTRY_BIRTH,ethnicity,...,ETHNICITY_CORRECT_ME,ETHNICITY_MOTHER,ETHNICITY_CORRECT_MOTHER,ETHNICITY_FATHER,ETHNICITY_CORRECT_FATHER,IMPUTED_ETHNIC_CB,IMPUTED_ETHNIC2_CB,ETHNIC_FR_CB,NGP_COUNTRY_INTEREST,Population
0,11135473,PHASE A,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,WHITE / EUROPEAN,...,NA,NA,NA,NA,NA,WHITE / EUROPEAN,WHITE,ENGLISH CANADIAN,4,CANADA
1,11118917,PHASE A,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,WHITE / EUROPEAN,...,white,NA,NA,NA,NA,WHITE / EUROPEAN,WHITE,FRENCH CANADIAN,4,CANADA
3,11129743,PHASE A,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,WHITE / EUROPEAN,...,white,NA,NA,NA,NA,WHITE / EUROPEAN,WHITE,FRENCH CANADIAN,4,CANADA
4,11131652,PHASE A,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,WHITE / EUROPEAN,...,NA,NA,NA,NA,NA,WHITE / EUROPEAN,WHITE,FRENCH CANADIAN,4,CANADA
5,11115717,PHASE A,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,WHITE / EUROPEAN,...,white,NA,NA,NA,NA,WHITE / EUROPEAN,WHITE,FRENCH CANADIAN,4,CANADA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29344,11124022,PHASE A,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,WHITE / EUROPEAN,...,white,NA,NA,NA,NA,WHITE / EUROPEAN,WHITE,FRENCH CANADIAN,4,CANADA
29345,11126766,PHASE A,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,WHITE / EUROPEAN,...,NA,NA,NA,NA,NA,WHITE / EUROPEAN,WHITE,FRENCH CANADIAN,4,CANADA
29346,11129199,PHASE B - SECOND WAVE,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,WHITE / EUROPEAN,...,white,WHITE,white,WHITE,white,NA,NA,NA,4,CANADA
29347,11137884,PHASE A,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,CANADA,WHITE / EUROPEAN,...,white,NA,NA,NA,NA,WHITE / EUROPEAN,WHITE,FRENCH CANADIAN,4,CANADA


In [12]:
data_med[data_med.Population=="CANADA"].groupby("ethnic_fr").ETHNICITY_ME.value_counts()

ethnic_fr         ETHNICITY_ME   
ENGLISH CANADIAN  WHITE                151
                  NA                    83
                  OTHER                  1
                  WHITEABORIGINAL        1
                  WHITEBLACK             1
                  WHITEJEWISH            1
FRENCH CANADIAN   WHITE              14086
                  NA                  4745
                  OTHER                 19
                  WHITEABORIGINAL       17
                  ABORIGINAL            12
                  BLACK                  4
                  WHITEOTHER             2
                  LATINO                 1
                  WHITEARAB              1
MISSING           WHITE                123
                  WHITEABORIGINAL        5
                  MISSING                2
                  ABORIGINAL             1
OTHER             WHITE                734
                  NA                   372
                  WHITEABORIGINAL      135
                  MI

In [40]:
other_aboriginal.ETHNICITY_ME.value_counts()

WHITEABORIGINAL    140
WHITE               50
NA                  49
ABORIGINAL          38
OTHER                3
BLACK                1
Name: ETHNICITY_ME, dtype: int64

Adding the Latino ancestry individuals

In [14]:
filters_lat=['PERU','COLOMBIA','MEXICO','CHILE']
data_latino = f.filter_by_value(data,filters_lat)

In [15]:
# Find columns with 'MOTHERS' or 'FATHERS' in their names
filtered_columns = [col for col in data_all.columns if 'MOTHERS' in col or 'FATHERS' in col]

data_latino['NGP_COUNTRY_INTEREST'] = data_latino[filtered_columns].isin(filters_lat).sum(axis=1)
print("Total individuals extracted with Latinx ancestry ", len(data_latino))

# Extract individuals with total ancestry
data_latino = data_latino[data_latino.NGP_COUNTRY_INTEREST == 4].copy()
data_latino['Population'] = data_latino.MOTHERS_MOTHER_COUNTRY_BIRTH

print("Total number of latinx individuals retained: ", len(data_latino))

print(data_latino.Population.value_counts())

Total individuals extracted with Latinx ancestry  326
Total number of latinx individuals retained:  244
COLOMBIA    100
MEXICO       55
CHILE        45
PERU         44
Name: Population, dtype: int64


In [16]:
data_latino.groupby("Population").ETHNICITY_ME.value_counts()

Population  ETHNICITY_ME
CHILE       LATINO          20
            NA              20
            WHITELATINO      4
            MISSING          1
COLOMBIA    LATINO          57
            NA              39
            WHITE            3
            WHITELATINO      1
MEXICO      LATINO          33
            NA              21
            MISSING          1
PERU        LATINO          26
            NA              16
            ABORIGINAL       2
Name: ETHNICITY_ME, dtype: int64

In [28]:
Perubians = data_latino[data_latino.Population == "PERU"]
Mexicans = data_latino[data_latino.Population == "MEXICO"].sample(50)

Add Europeans UK-FRANCE ancestry

In [17]:
filters_EUR=['FRANCE','UNITED_KINGDOM']
data_eur= f.filter_by_value(data,filters_EUR)

In [18]:
# Find columns with 'MOTHERS' or 'FATHERS' in their names
filtered_columns = [col for col in data_all.columns if 'MOTHERS' in col or 'FATHERS' in col]

data_eur['NGP_COUNTRY_INTEREST'] = data_eur[filtered_columns].isin(filters_EUR).sum(axis=1)
print("Total individuals extracted with French-UK ancestry ", len(data_eur))

# Extract individuals with total ancestry
data_eur = data_eur[data_eur.NGP_COUNTRY_INTEREST == 4].copy()
data_eur['Population'] = data_eur.MOTHERS_MOTHER_COUNTRY_BIRTH

print("Total number of EUR individuals retained: ", len(data_eur))

print(data_eur.Population.value_counts())

Total individuals extracted with French-UK ancestry  1818
Total number of EUR individuals retained:  560
FRANCE            419
UNITED_KINGDOM    141
Name: Population, dtype: int64


Add African ancestry

In [20]:
filters_AFR=['CAMEROON','CONGO_DEMOCRATIC_REPUBLIC','COTE_DIVORE']
data_AFR= f.filter_by_value(data,filters_AFR)

In [21]:
# Find columns with 'MOTHERS' or 'FATHERS' in their names
filtered_columns = [col for col in data_all.columns if 'MOTHERS' in col or 'FATHERS' in col]

data_AFR['NGP_COUNTRY_INTEREST'] = data_AFR[filtered_columns].isin(filters_AFR).sum(axis=1)
print("Total individuals extracted with AFR ancestry ", len(data_AFR))

# Extract individuals with total ancestry
data_AFR = data_AFR[data_AFR.NGP_COUNTRY_INTEREST == 4].copy()
data_AFR['Population'] = data_AFR.MOTHERS_MOTHER_COUNTRY_BIRTH

print("Total number of AFR individuals retained: ", len(data_AFR))

print(data_AFR.Population.value_counts())

Total individuals extracted with AFR ancestry  64
Total number of AFR individuals retained:  47
CONGO_DEMOCRATIC_REPUBLIC    25
CAMEROON                     22
Name: Population, dtype: int64


Write Keep file to extract from plink

In [ ]:
filters_EUR=['FRANCE','UNITED_KINGDOM']
data_eur= f.filter_by_value(data,filters_EUR)

In [ ]:
# Find columns with 'MOTHERS' or 'FATHERS' in their names
filtered_columns = [col for col in data_all.columns if 'MOTHERS' in col or 'FATHERS' in col]

data_eur['NGP_COUNTRY_INTEREST'] = data_eur[filtered_columns].isin(filters_EUR).sum(axis=1)
print("Total individuals extracted with French-UK ancestry ", len(data_eur))

# Extract individuals with total ancestry
data_eur = data_eur[data_eur.NGP_COUNTRY_INTEREST == 4].copy()
data_eur['Population'] = data_eur.MOTHERS_MOTHER_COUNTRY_BIRTH

print("Total number of EUR individuals retained: ", len(data_eur))

print(data_eur.Population.value_counts())

Total individuals extracted with French-UK ancestry  1818
Total number of EUR individuals retained:  560
FRANCE            419
UNITED_KINGDOM    141
Name: Population, dtype: int64


In [28]:
sample_EUR = data_eur.sample(200)

In [29]:
sample_EUR.Population.value_counts()

FRANCE            148
UNITED_KINGDOM     52
Name: Population, dtype: int64

In [25]:
sample_CaG = data_med.sample(500)

In [32]:
sample_CaG.ETHNICITY_ME.value_counts()

WHITE              363
NA                 132
WHITEABORIGINAL      2
ABORIGINAL           2
MISSING              1
Name: ETHNICITY_ME, dtype: int64

In [31]:
panel = pd.concat([sample_CaG,sample_EUR,data_latino,data_AFR])
f.write_keep("/home/gfemer/storage/Cartagene/Flagship/French_canadian/panel/gsa_merge_hg38.FCpanel.keep", panel)

Check ethnicity values of sampled CANADA

In [38]:
sample_ids = pd.read_table("/home/gfemer/storage/Cartagene/Flagship/French_canadian/panel/gsa_merge_hg38.FCpanel.keep",names=["COUNTRY","file111","file_111"],dtype=str)

In [39]:
ids = sample_ids[sample_ids.COUNTRY=="CANADA"]['file111'].tolist()

In [40]:
data.file111.isin(ids).sum()

500

In [46]:
data[data.file111.isin(ids)].Cregion.value_counts()

Montréal          332
Québec             90
Saguenay           29
Sherbrooke         21
Trois-Rivières     15
Gatineau           13
Name: Cregion, dtype: int64

check IDs that have almost 100% of the 4th component

In [5]:
ids_file = "/home/gfemer/storage/Cartagene/Flagship/French_canadian/admixture_results/ids_c4_k4.txt"
with open(ids_file, 'r') as file:
    ids = [line.strip('\n') for line in file]

In [12]:
data[data.file111.isin(ids)].Cregion.value_counts()

Saguenay          19
Montréal          15
Québec            10
Gatineau           1
Trois-Rivières     1
Name: Cregion, dtype: int64

outdated sampling, not used anymore

In [ ]:
FC_aboriginal = data_med[data_med.ethnic_fr=="FRENCH CANADIAN"]
FC_aboriginal = f.filter_by_value(FC_aboriginal,["WHITEABORIGINAL","ABORIGINAL"])

other_aboriginal = data_med[data_med["ethnic_fr"].isin(["OTHER","MISSING"])]
other_aboriginal = f.filter_by_value(other_aboriginal,["ABORIGINAL","WHITEABORIGINAL"])

In [ ]:
#Sample French Canadian "aboriginal"
FC_ab_sample = pd.concat([FC_aboriginal,other_aboriginal]).sample(50)
FC_ab_sample.Population = np.repeat("French_Canadian_Ab", 50, axis=0)

#Sample French Canadian "white"
FC_sample = f.sample_pop(data_med[data_med.ethnic_fr=="FRENCH CANADIAN"][data_med.ETHNICITY_ME == "WHITE"], samplepops=["FRENCH CANADIAN"], samplen=50, avoid=["ABORIGINAL"])
FC_sample.Population = np.repeat("French_Canadian",50)

#Sample English Canadian
EC_sample = data_med[data_med.ethnic_fr=="ENGLISH CANADIAN"].sample(50)
EC_sample.Population = np.repeat("English_Canadian",50)

#Sample French and British
FR_sample = data_med[data_med.Population=="FRANCE"].sample(50)
UK_sample = data_med[data_med.Population=="UNITED_KINGDOM"].sample(50)

/tmp/ipykernel_719663/14898016.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  FC_sample = f.sample_pop(data_med[data_med.ethnic_fr=="FRENCH CANADIAN"][data_med.ETHNICITY_ME == "WHITE"], samplepops=["FRENCH CANADIAN"], samplen=50, avoid=["ABORIGINAL"])


In [ ]:
other_aboriginal[other_aboriginal["ETHNICITY_ME"].isin(["WHITEABORIGINAL","ABORIGINAL"])][["ETHNICITY_ME","ETHNICITY_MOTHER","ETHNICITY_FATHER"]].value_counts()

ETHNICITY_ME     ETHNICITY_MOTHER  ETHNICITY_FATHER
WHITEABORIGINAL  WHITEABORIGINAL   WHITE               46
                 WHITE             WHITEABORIGINAL     40
                 WHITEABORIGINAL   WHITEABORIGINAL     32
ABORIGINAL       ABORIGINAL        ABORIGINAL          12
                 WHITE             ABORIGINAL          10
                 NA                NA                   9
WHITEABORIGINAL  WHITE             WHITE                6
ABORIGINAL       ABORIGINAL        WHITE                5
WHITEABORIGINAL  ABORIGINAL        WHITE                5
                 NA                NA                   5
                 WHITE             ABORIGINAL           4
ABORIGINAL       ABORIGINAL        MISSING              1
                 MISSING           ABORIGINAL           1
WHITEABORIGINAL  WHITE             OTHER                1
                 WHITEABORIGINAL   MISSING              1
dtype: int64

In [ ]:
FC_aboriginal

WHITE              59
WHITEABORIGINAL    17
ABORIGINAL         12
Name: ETHNICITY_ME, dtype: int64

In [29]:
panel = pd.concat([EC_sample,FC_ab_sample,FC_sample,UK_sample,FR_sample,Perubians,Mexicans])

In [31]:
f.write_keep("/home/gfemer/storage/Cartagene/Flagship/French_canadian/panel/gsa_merge_hg38.FCpanel_UKFRLat.keep", panel)

/home/gfemer/storage/Cartagene/Flagship/French_canadian/panel/gsa_merge_hg38.FCpanel_UKFRLat.keep already exists, overwritting
